In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "2"


from torchvision.utils import make_grid, save_image
import torchvision.models as models
import torch.nn.functional as F
import matplotlib.pyplot as plt
from copy import deepcopy
import numpy as np
import random
import torch
import json
import PIL


from handler import geozoom_handler
# from helpers import *
from fc_net import *
from utils import *
from sa import *

In [2]:
# random.sample(range(0, len(image_names)), 5)

In [3]:
# image_names = get_png_names("../pooling/data/MEX2/")#[100:105]
image_names = get_png_names("../pooling/data/MEX2/")
# image_indices = random.sample(range(0, len(image_names)), 1000)
# image_names = [image_names[i] for i in range(len(image_names)) if i in image_indices]
y = get_migrants("../pooling/data/migration_data.json" , image_names)

1779 municipalities.


In [4]:
# for i in image_names:
#     print(load_inputs(i).shape)

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet18 = models.resnet18().to(device)
attn_model = attnNet(resnet = resnet18).to(device)
lr = .00001
criterion = torch.nn.L1Loss(reduction = 'mean')
attn_optimizer = torch.optim.Adam(attn_model.parameters(), lr = lr)

fc_model = fc_net(resnet = resnet18).to(device)
fc_optimizer = torch.optim.Adam(fc_model.parameters(), lr = .01)

butler = geozoom_handler(attn_model, 
                         fc_model, device, 
                         criterion, 
                         attn_optimizer, 
                         fc_optimizer, 
                         num_thresholds = 10,
                         reduction_percent = .90,
                         convergence_dims = (358, 284),
                         plot = False, 
                         v = False)

In [6]:
attn_model.sa

SelfAttention(
  (query_conv): Conv2d(256, 32, kernel_size=(1, 1), stride=(1, 1))
  (key_conv): Conv2d(256, 32, kernel_size=(1, 1), stride=(1, 1))
  (value_conv): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
  (softmax): Softmax(dim=-1)
)

In [7]:
BATCH_SIZE = 1
SPLIT = .60

train_dl, val_dl = butler.prep_attn_data(image_names, y, SPLIT, BATCH_SIZE)

In [8]:
# for i,o in val_dl:
#     print(load_inputs(i[0]).shape, o)

In [ ]:
butler.train_attn_model(train_dl, val_dl)

Epoch:  0
  Training Loss:  1672.4846821735284
  Validation Loss:  2069.351448468278
Loss thresholds for training:  [0.0, 206.93514484682777, 413.87028969365554, 620.8054345404832, 827.7405793873111, 1034.675724234139, 1241.6108690809665, 1448.5460139277943, 1655.4811587746221, 1862.41630362145]
Starting from threshold:  9  with value:  1862.41630362145


Epoch:  1
  Training Loss:  1665.7875354511223
  Validation Loss:  2065.127903135975


Epoch:  2
  Training Loss:  1660.1707047343589
  Validation Loss:  2061.178655219882


Epoch:  3
  Training Loss:  1654.6936256619626
  Validation Loss:  2056.690129456895


Epoch:  4
  Training Loss:  1649.2981030983465
  Validation Loss:  2052.851726277491


Epoch:  5
  Training Loss:  1643.849188456942
  Validation Loss:  2046.4678743340996


Epoch:  6
  Training Loss:  1638.370018248482
  Validation Loss:  2045.651656188322


Epoch:  7
  Training Loss:  1633.1199393518193
  Validation Loss:  2038.7336161886708


Epoch:  8
  Training Loss:  1628.

/opt/conda/envs/rapids/lib/python3.7/site-packages/torch/nn/functional.py:3328: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")




Epoch:  52
  Training Loss:  1496.3676218535147
  Validation Loss:  1878.3499095788163


Epoch:  53
  Training Loss:  1492.160451258022
  Validation Loss:  1888.7982713185


Epoch:  54
  Training Loss:  1488.5718308639168
  Validation Loss:  1892.70490966754


Epoch:  55
  Training Loss:  1484.1975531394837
  Validation Loss:  1916.4308938765794


Epoch:  56
  Training Loss:  1482.456878254541
  Validation Loss:  1914.2083309259308


Epoch:  57
  Training Loss:  1479.3540918552217
  Validation Loss:  1872.4348036733907


Epoch:  58
  Training Loss:  1476.4430887853307
  Validation Loss:  1870.9246898715417


Epoch:  59
  Training Loss:  1473.7333079287127
  Validation Loss:  1821.5050234848195


Epoch:  60
  Training Loss:  1472.3028099253117
  Validation Loss:  1885.0380153870315


Epoch:  61
  Training Loss:  1469.0364369014321
  Validation Loss:  1862.9745060352798


Epoch:  62
  Training Loss:  1468.2420258598006
  Validation Loss:  1875.4539187570635


Epoch:  63
  Training Loss